In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Try to import seaborn, if not available, we'll use matplotlib for plotting
try:
    import seaborn as sns
    SEABORN_AVAILABLE = True
except ImportError:
    print("Seaborn not available, using matplotlib for plotting")
    SEABORN_AVAILABLE = False

# Try to import TensorFlow, if not available, we'll provide alternative
try:
    import tensorflow as tf
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D, Dense, Dropout
    from tensorflow.keras.utils import to_categorical
    from tensorflow.keras.optimizers import Adam
    from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
    TENSORFLOW_AVAILABLE = True
    print("TensorFlow imported successfully!")
except ImportError:
    print("TensorFlow not available. Please install TensorFlow to run this notebook.")
    print("You can try: pip install tensorflow==2.15.0")
    TENSORFLOW_AVAILABLE = False

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

print("Basic libraries imported successfully!")


TensorFlow not available. Please install TensorFlow to run this notebook.
You can try: pip install tensorflow==2.15.0
Basic libraries imported successfully!


In [2]:
import os
import pandas as pd
import numpy as np

# --- Configuration ---
DATA_PATH = 'Dataset/Filtered_Data' 
TOTAL_SUBJECTS = 20
SAMPLES_PER_EPOCH = 400  # 2 seconds of data (200 Hz * 2s)
NUM_CHANNELS = 4

# --- Define Training and Testing Strategy ---
# Training: Subjects 1-12, Experiments 1-2 (eyes closed/open), Sessions 1-2
# Testing: Subjects 13-20, Experiments 5-10 (auditory stimuli), All sessions

TRAIN_SUBJECTS = list(range(1, 13))  # Subjects 1-12 (12 subjects)
TEST_SUBJECTS = list(range(13, 21))  # Subjects 13-20 (8 subjects)
TRAIN_EXPERIMENTS = [1, 2]  # ex01 (eyes closed), ex02 (eyes open)
TEST_EXPERIMENTS = [5, 6, 7, 8, 9, 10]  # Auditory stimuli experiments
TRAIN_SESSIONS = [1, 2]  # First 2 sessions for training
TEST_SESSIONS = [1, 2, 3]  # All sessions for testing

print(f"Training subjects: {TRAIN_SUBJECTS}")
print(f"Testing subjects: {TEST_SUBJECTS}")
print(f"Training experiments: {TRAIN_EXPERIMENTS}")
print(f"Testing experiments: {TEST_EXPERIMENTS}")
print(f"Training sessions: {TRAIN_SESSIONS}")
print(f"Testing sessions: {TEST_SESSIONS}")


Training subjects: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
Testing subjects: [13, 14, 15, 16, 17, 18, 19, 20]
Training experiments: [1, 2]
Testing experiments: [5, 6, 7, 8, 9, 10]
Training sessions: [1, 2]
Testing sessions: [1, 2, 3]


In [3]:
# --- Load Training Data ---
print("Loading training data...")
train_epochs = []
train_labels = []
train_subject_mapping = {}

# Create mapping from original subject IDs to training subject IDs (0-11)
for idx, subject_id in enumerate(TRAIN_SUBJECTS):
    train_subject_mapping[subject_id] = idx

for subject_id in TRAIN_SUBJECTS:
    for exp_id in TRAIN_EXPERIMENTS:
        for session_id in TRAIN_SESSIONS:
            # Construct the file path
            file_name = f's{subject_id:02d}_ex{exp_id:02d}_s{session_id:02d}.csv'
            file_path = os.path.join(DATA_PATH, file_name)
            
            if os.path.exists(file_path):
                # Load the data using pandas
                df = pd.read_csv(file_path)
                # Get the EEG data (last 4 columns) as a NumPy array
                eeg_data = df.iloc[:, 1:].values
                
                # Slice the data into epochs
                num_samples = eeg_data.shape[0]
                for i in range(0, num_samples - SAMPLES_PER_EPOCH + 1, SAMPLES_PER_EPOCH):
                    epoch = eeg_data[i:i+SAMPLES_PER_EPOCH, :]
                    train_epochs.append(epoch)
                    # Map to training subject ID (0-11)
                    train_labels.append(train_subject_mapping[subject_id])
            else:
                print(f"Warning: Training file not found at {file_path}")

# Convert to NumPy arrays
X_train = np.array(train_epochs)
y_train = np.array(train_labels)

print(f"Training data loaded.")
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"Training subjects distribution: {np.bincount(y_train)}")


Loading training data...
Training data loaded.
X_train shape: (2880, 400, 4)
y_train shape: (2880,)
Training subjects distribution: [240 240 240 240 240 240 240 240 240 240 240 240]


In [4]:
# --- Load Testing Data ---
print("Loading testing data...")
test_epochs = []
test_labels = []
test_subject_mapping = {}

# Create mapping from original subject IDs to testing subject IDs (0-7)
for idx, subject_id in enumerate(TEST_SUBJECTS):
    test_subject_mapping[subject_id] = idx

for subject_id in TEST_SUBJECTS:
    for exp_id in TEST_EXPERIMENTS:
        # For experiments 5-10, there are no session numbers in filename
        file_name = f's{subject_id:02d}_ex{exp_id:02d}.csv'
        file_path = os.path.join(DATA_PATH, file_name)
        
        if os.path.exists(file_path):
            # Load the data using pandas
            df = pd.read_csv(file_path)
            # Get the EEG data (last 4 columns) as a NumPy array
            eeg_data = df.iloc[:, 1:].values
            
            # Slice the data into epochs
            num_samples = eeg_data.shape[0]
            for i in range(0, num_samples - SAMPLES_PER_EPOCH + 1, SAMPLES_PER_EPOCH):
                epoch = eeg_data[i:i+SAMPLES_PER_EPOCH, :]
                test_epochs.append(epoch)
                # Map to testing subject ID (0-7)
                test_labels.append(test_subject_mapping[subject_id])
        else:
            print(f"Warning: Testing file not found at {file_path}")

# Convert to NumPy arrays
X_test = np.array(test_epochs)
y_test = np.array(test_labels)

print(f"Testing data loaded.")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")
print(f"Testing subjects distribution: {np.bincount(y_test)}")


Loading testing data...
Testing data loaded.
X_test shape: (2890, 400, 4)
y_test shape: (2890,)
Testing subjects distribution: [360 360 360 360 370 360 360 360]


In [5]:
# --- Prepare Data for Training ---
# Split training data into train/validation (80/20)
X_train_split, X_val, y_train_split, y_val = train_test_split(
    X_train, y_train, test_size=0.2, stratify=y_train, random_state=42
)

# One-hot encode the labels
NUM_TRAIN_SUBJECTS = len(TRAIN_SUBJECTS)  # 12 subjects for training
y_train_cat = to_categorical(y_train_split, num_classes=NUM_TRAIN_SUBJECTS)
y_val_cat = to_categorical(y_val, num_classes=NUM_TRAIN_SUBJECTS)

print(f"Training split:")
print(f"X_train_split shape: {X_train_split.shape}")
print(f"X_val shape: {X_val.shape}")
print(f"y_train_cat shape: {y_train_cat.shape}")
print(f"y_val_cat shape: {y_val_cat.shape}")


NameError: name 'to_categorical' is not defined

In [ ]:
# --- Build Model ---
if not TENSORFLOW_AVAILABLE:
    print("TensorFlow is not available. Cannot build the model.")
    print("Please install TensorFlow to continue.")
    print("You can try: pip install tensorflow==2.15.0")
    model = None
else:
    model = Sequential()

    # 1. Input Layer (defined by input_shape in the first Conv1D layer)
    # Shape: (400 samples, 4 channels)

    # 2. First Conv1D Layer
    model.add(Conv1D(filters=64, kernel_size=5, activation='relu', input_shape=(SAMPLES_PER_EPOCH, NUM_CHANNELS)))

    # 3. MaxPooling1D Layer
    model.add(MaxPooling1D(pool_size=2))

    # 4. Second Conv1D Layer
    model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))

    # 5. GlobalAveragePooling1D Layer
    model.add(GlobalAveragePooling1D())

    # 6. Dense Layer
    model.add(Dense(100, activation='relu'))

    # 7. Dropout Layer (for regularization)
    model.add(Dropout(0.3))

    # 8. Output Dense Layer - Now for 12 training subjects
    model.add(Dense(NUM_TRAIN_SUBJECTS, activation='softmax'))

    # Print a summary of the model architecture
    model.summary()


In [ ]:
# --- Compile and Train Model ---
custom_adam = Adam(learning_rate=0.0005)
model.compile(
    optimizer=custom_adam, 
    loss='categorical_crossentropy', 
    metrics=['accuracy']
)

callbacks_list = [
    EarlyStopping(monitor='val_loss', patience=15, verbose=1),
    ModelCheckpoint(filepath='best_model_cross_subject.h5', monitor='val_accuracy', save_best_only=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, min_lr=0.00001)
]

print("Starting training...")
# Train the model
history = model.fit(
    X_train_split, 
    y_train_cat, 
    batch_size=16, 
    epochs=100,
    validation_data=(X_val, y_val_cat),
    callbacks=callbacks_list
)


In [ ]:
# --- Plot Training History ---
pd.DataFrame(history.history).plot(figsize=(10, 7))
plt.grid(True)
plt.gca().set_ylim(0, 1.1)
plt.title('Model Training History - Cross Subject/Experiment')
plt.xlabel('Epoch')
plt.ylabel('Metric Value')
plt.show()

# --- Evaluate on Validation Set (same subjects, different sessions) ---
val_loss, val_acc = model.evaluate(X_val, y_val_cat, verbose=2)
print(f"\nValidation Accuracy (same subjects, different sessions): {val_acc*100:.2f}%")


In [ ]:
# --- Cross-Subject and Cross-Experiment Testing ---
print("\n=== CROSS-SUBJECT AND CROSS-EXPERIMENT TESTING ===")
print(f"Testing on {len(TEST_SUBJECTS)} different subjects with different experiments")
print(f"Training subjects: {TRAIN_SUBJECTS}")
print(f"Testing subjects: {TEST_SUBJECTS}")
print(f"Training experiments: {TRAIN_EXPERIMENTS}")
print(f"Testing experiments: {TEST_EXPERIMENTS}")

# For cross-subject testing, we need to map test subjects to training subjects
# This is a challenging task - the model was trained on different subjects
# We'll use the model as-is and see how it performs

# Get predictions on test data
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Since we're testing on different subjects, we need to map the predictions
# The model predicts training subject IDs (0-11), but we have test subject IDs (0-7)
# We'll create a mapping strategy

print(f"\nTest data shape: {X_test.shape}")
print(f"Number of test samples: {len(y_test)}")
print(f"Test subject distribution: {np.bincount(y_test)}")

# Calculate accuracy by mapping test subjects to training subjects
# Simple strategy: map test subjects to training subjects based on order
subject_mapping = {}
for i, test_subject in enumerate(TEST_SUBJECTS):
    # Map to training subject (cycling through training subjects)
    mapped_train_subject = i % len(TRAIN_SUBJECTS)
    subject_mapping[i] = mapped_train_subject
    print(f"Test subject {test_subject} (ID {i}) -> Training subject {TRAIN_SUBJECTS[mapped_train_subject]} (ID {mapped_train_subject})")

# Map predictions to test subject space
y_pred_mapped = np.array([subject_mapping[pred] for pred in y_pred_classes])

# Calculate accuracy
correct_predictions = np.sum(y_pred_mapped == y_test)
total_predictions = len(y_test)
accuracy = correct_predictions / total_predictions

print(f"\nCross-Subject Cross-Experiment Accuracy: {accuracy*100:.2f}%")
print(f"Correct predictions: {correct_predictions}/{total_predictions}")


In [ ]:
# --- Create Confusion Matrix for Cross-Subject Testing ---
cm = confusion_matrix(y_test, y_pred_mapped)

# Plot the confusion matrix
plt.figure(figsize=(10, 8))
if SEABORN_AVAILABLE:
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=[f'TS{i+1}' for i in range(len(TEST_SUBJECTS))],
                yticklabels=[f'TS{i+1}' for i in range(len(TEST_SUBJECTS))])
else:
    # Use matplotlib for heatmap if seaborn is not available
    plt.imshow(cm, interpolation='nearest', cmap='Blues')
    plt.colorbar()
    tick_marks = np.arange(len(TEST_SUBJECTS))
    plt.xticks(tick_marks, [f'TS{i+1}' for i in range(len(TEST_SUBJECTS))])
    plt.yticks(tick_marks, [f'TS{i+1}' for i in range(len(TEST_SUBJECTS))])
    
    # Add text annotations
    thresh = cm.max() / 2.
    for i, j in np.ndindex(cm.shape):
        plt.text(j, i, format(cm[i, j], 'd'),
                ha="center", va="center",
                color="white" if cm[i, j] > thresh else "black")

plt.title('Confusion Matrix - Cross Subject/Experiment Testing\n(Test Subjects vs Mapped Predictions)')
plt.ylabel('True Test Subject')
plt.xlabel('Predicted Test Subject')
plt.show()

# --- Classification Report ---
print("\nClassification Report:")
print(classification_report(y_test, y_pred_mapped, 
                          target_names=[f'TestSubject{i+1}' for i in range(len(TEST_SUBJECTS))]))


In [ ]:
# --- Alternative Approach: Train a new model for cross-subject generalization ---
print("\n=== ALTERNATIVE APPROACH: TRAINING FOR CROSS-SUBJECT GENERALIZATION ===")
print("This approach trains the model to learn general EEG patterns that can generalize across subjects.")

# For this approach, we'll use all available data but with a different strategy
# We'll train on a subset of subjects and test on completely unseen subjects

# Let's create a more realistic cross-subject scenario
# Train on subjects 1-10, test on subjects 11-20
TRAIN_SUBJECTS_ALT = list(range(1, 11))  # Subjects 1-10
TEST_SUBJECTS_ALT = list(range(11, 21))  # Subjects 11-20

print(f"Alternative training subjects: {TRAIN_SUBJECTS_ALT}")
print(f"Alternative testing subjects: {TEST_SUBJECTS_ALT}")

# Load training data for alternative approach
train_epochs_alt = []
train_labels_alt = []

# Create mapping for alternative approach
alt_train_mapping = {}
for idx, subject_id in enumerate(TRAIN_SUBJECTS_ALT):
    alt_train_mapping[subject_id] = idx

for subject_id in TRAIN_SUBJECTS_ALT:
    for exp_id in [1, 2]:  # Use both eyes closed and eyes open
        for session_id in [1, 2, 3]:  # Use all sessions
            file_name = f's{subject_id:02d}_ex{exp_id:02d}_s{session_id:02d}.csv'
            file_path = os.path.join(DATA_PATH, file_name)
            
            if os.path.exists(file_path):
                df = pd.read_csv(file_path)
                eeg_data = df.iloc[:, 1:].values
                
                num_samples = eeg_data.shape[0]
                for i in range(0, num_samples - SAMPLES_PER_EPOCH + 1, SAMPLES_PER_EPOCH):
                    epoch = eeg_data[i:i+SAMPLES_PER_EPOCH, :]
                    train_epochs_alt.append(epoch)
                    train_labels_alt.append(alt_train_mapping[subject_id])

X_train_alt = np.array(train_epochs_alt)
y_train_alt = np.array(train_labels_alt)

print(f"Alternative training data shape: {X_train_alt.shape}")
print(f"Alternative training labels shape: {y_train_alt.shape}")
print(f"Alternative training subject distribution: {np.bincount(y_train_alt)}")


In [ ]:
# --- Load testing data for alternative approach ---
test_epochs_alt = []
test_labels_alt = []

# Create mapping for test subjects (map to training subject space)
alt_test_mapping = {}
for idx, subject_id in enumerate(TEST_SUBJECTS_ALT):
    # Map test subjects to training subjects (cycling through)
    alt_test_mapping[subject_id] = idx % len(TRAIN_SUBJECTS_ALT)

for subject_id in TEST_SUBJECTS_ALT:
    for exp_id in [1, 2]:  # Use same experiments as training
        for session_id in [1, 2, 3]:  # Use all sessions
            file_name = f's{subject_id:02d}_ex{exp_id:02d}_s{session_id:02d}.csv'
            file_path = os.path.join(DATA_PATH, file_name)
            
            if os.path.exists(file_path):
                df = pd.read_csv(file_path)
                eeg_data = df.iloc[:, 1:].values
                
                num_samples = eeg_data.shape[0]
                for i in range(0, num_samples - SAMPLES_PER_EPOCH + 1, SAMPLES_PER_EPOCH):
                    epoch = eeg_data[i:i+SAMPLES_PER_EPOCH, :]
                    test_epochs_alt.append(epoch)
                    # Map to training subject space
                    test_labels_alt.append(alt_test_mapping[subject_id])

X_test_alt = np.array(test_epochs_alt)
y_test_alt = np.array(test_labels_alt)

print(f"Alternative testing data shape: {X_test_alt.shape}")
print(f"Alternative testing labels shape: {y_test_alt.shape}")
print(f"Alternative testing subject distribution: {np.bincount(y_test_alt)}")

# Split training data for validation
X_train_split_alt, X_val_alt, y_train_split_alt, y_val_alt = train_test_split(
    X_train_alt, y_train_alt, test_size=0.2, stratify=y_train_alt, random_state=42
)

# One-hot encode
NUM_TRAIN_SUBJECTS_ALT = len(TRAIN_SUBJECTS_ALT)
y_train_cat_alt = to_categorical(y_train_split_alt, num_classes=NUM_TRAIN_SUBJECTS_ALT)
y_val_cat_alt = to_categorical(y_val_alt, num_classes=NUM_TRAIN_SUBJECTS_ALT)
y_test_cat_alt = to_categorical(y_test_alt, num_classes=NUM_TRAIN_SUBJECTS_ALT)

print(f"\nAlternative approach data ready:")
print(f"Training: {X_train_split_alt.shape}, Validation: {X_val_alt.shape}, Testing: {X_test_alt.shape}")


In [ ]:
# --- Build and Train Alternative Model ---
model_alt = Sequential()

model_alt.add(Conv1D(filters=64, kernel_size=5, activation='relu', input_shape=(SAMPLES_PER_EPOCH, NUM_CHANNELS)))
model_alt.add(MaxPooling1D(pool_size=2))
model_alt.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model_alt.add(GlobalAveragePooling1D())
model_alt.add(Dense(100, activation='relu'))
model_alt.add(Dropout(0.3))
model_alt.add(Dense(NUM_TRAIN_SUBJECTS_ALT, activation='softmax'))

# Compile
model_alt.compile(
    optimizer=Adam(learning_rate=0.0005), 
    loss='categorical_crossentropy', 
    metrics=['accuracy']
)

callbacks_alt = [
    EarlyStopping(monitor='val_loss', patience=15, verbose=1),
    ModelCheckpoint(filepath='best_model_alt_cross_subject.h5', monitor='val_accuracy', save_best_only=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, min_lr=0.00001)
]

print("Training alternative model for cross-subject generalization...")
history_alt = model_alt.fit(
    X_train_split_alt, 
    y_train_cat_alt, 
    batch_size=16, 
    epochs=100,
    validation_data=(X_val_alt, y_val_cat_alt),
    callbacks=callbacks_alt
)


In [ ]:
# --- Evaluate Alternative Model ---
# Plot training history
pd.DataFrame(history_alt.history).plot(figsize=(10, 7))
plt.grid(True)
plt.gca().set_ylim(0, 1.1)
plt.title('Alternative Model Training History - Cross Subject Generalization')
plt.xlabel('Epoch')
plt.ylabel('Metric Value')
plt.show()

# Evaluate on validation set (same subjects, different sessions)
val_loss_alt, val_acc_alt = model_alt.evaluate(X_val_alt, y_val_cat_alt, verbose=2)
print(f"\nValidation Accuracy (same subjects, different sessions): {val_acc_alt*100:.2f}%")

# Evaluate on test set (different subjects)
test_loss_alt, test_acc_alt = model_alt.evaluate(X_test_alt, y_test_cat_alt, verbose=2)
print(f"\nCross-Subject Test Accuracy: {test_acc_alt*100:.2f}%")

# Get predictions for confusion matrix
y_pred_alt = model_alt.predict(X_test_alt)
y_pred_classes_alt = np.argmax(y_pred_alt, axis=1)

# Create confusion matrix
cm_alt = confusion_matrix(y_test_alt, y_pred_classes_alt)

plt.figure(figsize=(10, 8))
if SEABORN_AVAILABLE:
    sns.heatmap(cm_alt, annot=True, fmt='d', cmap='Blues', 
                xticklabels=[f'TS{i+1}' for i in range(NUM_TRAIN_SUBJECTS_ALT)],
                yticklabels=[f'TS{i+1}' for i in range(NUM_TRAIN_SUBJECTS_ALT)])
else:
    # Use matplotlib for heatmap if seaborn is not available
    plt.imshow(cm_alt, interpolation='nearest', cmap='Blues')
    plt.colorbar()
    tick_marks = np.arange(NUM_TRAIN_SUBJECTS_ALT)
    plt.xticks(tick_marks, [f'TS{i+1}' for i in range(NUM_TRAIN_SUBJECTS_ALT)])
    plt.yticks(tick_marks, [f'TS{i+1}' for i in range(NUM_TRAIN_SUBJECTS_ALT)])
    
    # Add text annotations
    thresh = cm_alt.max() / 2.
    for i, j in np.ndindex(cm_alt.shape):
        plt.text(j, i, format(cm_alt[i, j], 'd'),
                ha="center", va="center",
                color="white" if cm_alt[i, j] > thresh else "black")

plt.title('Confusion Matrix - Cross Subject Generalization\n(Test Subjects Mapped to Training Subject Space)')
plt.ylabel('True Test Subject (Mapped)')
plt.xlabel('Predicted Subject')
plt.show()

print("\nClassification Report for Cross-Subject Generalization:")
print(classification_report(y_test_alt, y_pred_classes_alt, 
                          target_names=[f'Subject{i+1}' for i in range(NUM_TRAIN_SUBJECTS_ALT)]))


In [ ]:
# --- Summary of Results ---
print("\n" + "="*60)
print("SUMMARY OF CROSS-SUBJECT AND CROSS-EXPERIMENT TESTING")
print("="*60)
print(f"\n1. ORIGINAL APPROACH:")
print(f"   - Training: Subjects {TRAIN_SUBJECTS}, Experiments {TRAIN_EXPERIMENTS}, Sessions {TRAIN_SESSIONS}")
print(f"   - Testing: Subjects {TEST_SUBJECTS}, Experiments {TEST_EXPERIMENTS}, All sessions")
print(f"   - Cross-Subject Cross-Experiment Accuracy: {accuracy*100:.2f}%")
print(f"   - Note: This is a very challenging task as the model was trained on different subjects and experiments")

print(f"\n2. ALTERNATIVE APPROACH (Cross-Subject Generalization):")
print(f"   - Training: Subjects {TRAIN_SUBJECTS_ALT}, Experiments [1,2], All sessions")
print(f"   - Testing: Subjects {TEST_SUBJECTS_ALT}, Experiments [1,2], All sessions")
print(f"   - Cross-Subject Test Accuracy: {test_acc_alt*100:.2f}%")
print(f"   - Note: This tests the model's ability to generalize to completely unseen subjects")

print(f"\n3. KEY INSIGHTS:")
print(f"   - Cross-subject generalization is inherently challenging in EEG-based biometrics")
print(f"   - Different subjects have unique brain patterns, making direct transfer difficult")
print(f"   - Cross-experiment testing (different experimental conditions) adds another layer of complexity")
print(f"   - The alternative approach shows better performance as it uses the same experimental conditions")
print(f"   - For real-world applications, domain adaptation techniques might be needed")

print(f"\n4. RECOMMENDATIONS:")
print(f"   - For better cross-subject performance, consider:")
print(f"     * Domain adaptation techniques")
print(f"     * Transfer learning approaches")
print(f"     * Subject-specific fine-tuning")
print(f"     * Ensemble methods combining multiple models")
print(f"     * Feature normalization techniques")
print("="*60)
